## 3가지 Boosting 실습

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
data = pd.read_csv('./data/otto_train.csv')
data.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [3]:
data = data.drop(['id'], axis = 1) # id 제거

In [4]:
mapping_dict = {"Class_1": 1,
                "Class_2": 2,
                "Class_3": 3,
                "Class_4": 4,
                "Class_5": 5,
                "Class_6": 6,
                "Class_7": 7,
                "Class_8": 8,
                "Class_9": 9}
after_mapping_target = data['target'].apply(lambda x: mapping_dict[x])

In [5]:
feature_columns = list(data.columns.difference(['target'])) # target을 제외한 모든 행
X = data[feature_columns] # 설명변수
y = after_mapping_target # 타겟변수
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(49502, 93) (12376, 93) (49502,) (12376,)


## 1. XGBoost

In [10]:
import xgboost as xgb
import time
from sklearn.metrics import accuracy_score

In [12]:
start = time.time()
xgb_dtrain = xgb.DMatrix(data=train_x, label=train_y) # xgb.DMatrix: 데이터를 xgboost 모델에 맞게 변환
xgb_dtest = xgb.DMatrix(data=test_x) # 동일 효과
xgb_params = {'max_depth': 10, # 트리 깊이
              'learning_rate': 0.01, # Step Size
              'n_estimators': 100, # 트리 생성 개수
              'objective': 'multi:softmax', # 목적 함수
              'num_class': len(set(train_y)) + 1}
xgb_model = xgb.train(params=xgb_params, dtrain=xgb_dtrain) # 학습 진행
xgb_model_predict = xgb_model.predict(xgb_dtest)
print("Accuracy: %.2f" % (accuracy_score(test_y, xgb_model_predict) * 100), "%")
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

[03:56:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[03:56:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 76.67 %
Time: 9.20 seconds


## 2. LightGBM

In [15]:
import lightgbm as lgb

In [16]:
start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 10, # 트리 깊이
             'learning_rate': 0.01, # Step Size
             'n_estimators': 100, # Number of trees, 트리 생성 개수
             'objective': 'multiclass', # 목적 함수
             'num_class': len(set(train_y)) + 1} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain)
lgb_model_predict = np.argmax(lgb_model.predict(test_x), axis = 1) # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, lgb_model_predict) * 100), "%")
print("Time: %.2f" % (time.time() - start), "seconds")

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030304 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3110
[LightGBM] [Info] Number of data points in the train set: 49502, number of used features: 93
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -3.476745
[LightGBM] [Info] Start training from score -1.341381
[LightGBM] [Info] Start training from score -2.039019
[LightGBM] [Info] Start training from score -3.135151
[LightGBM] [Info] Start training from score -3.125444
[LightGBM] [Info] Start training from score -1.481556
[Ligh

## 3. Catboost

In [19]:
import catboost as cb

In [21]:
start = time.time()
cb_dtrain = cb.Pool(data=train_x, label=train_y)
cb_params = {'max_depth': 10,
             'learning_rate': 0.01,
             'n_estimators': 100,
             'eval_metric': 'Accuracy', # 평가 척도
             'loss_function': 'MultiClass'} # 손실 함수, 목적 함수
cb_model = cb.train(pool=cb_dtrain, params=cb_params)
cb_model_predict = np.argmax(cb_model.predict(test_x), axis=1) + 1
print("Accuracy: %.2f" % (accuracy_score(test_y, cb_model_predict) * 100), "%")
print("Time: %.2f" % (time.time() - start), "seconds")

0:	learn: 0.5907034	total: 905ms	remaining: 1m 29s
1:	learn: 0.6356107	total: 1.93s	remaining: 1m 34s
2:	learn: 0.6411256	total: 2.82s	remaining: 1m 31s
3:	learn: 0.6480344	total: 3.65s	remaining: 1m 27s
4:	learn: 0.6508222	total: 4.48s	remaining: 1m 25s
5:	learn: 0.6499939	total: 5.26s	remaining: 1m 22s
6:	learn: 0.6507818	total: 6.01s	remaining: 1m 19s
7:	learn: 0.6548422	total: 6.7s	remaining: 1m 17s
8:	learn: 0.6559533	total: 7.44s	remaining: 1m 15s
9:	learn: 0.6560947	total: 8.17s	remaining: 1m 13s
10:	learn: 0.6568421	total: 8.94s	remaining: 1m 12s
11:	learn: 0.6588219	total: 9.76s	remaining: 1m 11s
12:	learn: 0.6592259	total: 10.8s	remaining: 1m 12s
13:	learn: 0.6611248	total: 11.9s	remaining: 1m 13s
14:	learn: 0.6625591	total: 12.7s	remaining: 1m 12s
15:	learn: 0.6631853	total: 13.5s	remaining: 1m 11s
16:	learn: 0.6639328	total: 14.3s	remaining: 1m 9s
17:	learn: 0.6668821	total: 15.1s	remaining: 1m 8s
18:	learn: 0.6669630	total: 15.9s	remaining: 1m 7s
19:	learn: 0.6675286	total

## Regression 목적 함수 사용

In [23]:
data = pd.read_csv("./data/kc_house_data.csv")

In [24]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1)

In [25]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [35]:
start = time.time()
lgb_dtrain = lgb.Dataset(data=train_x, label=train_y)
lgb_params = {'max_depth': 10,
              'learning_rate': 0.01,
              'n_estimator' : 100,
              'objective' : 'regression'} # 목적 함수 regression
lgb_model = lgb.train(params=lgb_params, train_set=lgb_dtrain)

[LightGBM] [Warning] Unknown parameter: n_estimator
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: n_estimator
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000517 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 237
[LightGBM] [Info] Number of data points in the train set: 15129, number of used features: 8
[LightGBM] [Info] Start training from score 537729.263666


In [39]:
from math import sqrt
from sklearn.metrics import mean_squared_error # RMSE
import random

In [40]:
sqrt(mean_squared_error(lgb_model.predict(test_x), test_y))

249111.2021710895